## Amount of money saved in NYC 2017

In [1]:
import pandas as pd
#from fbprophet import Prophet
from sklearn import metrics
from sklearn import preprocessing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import StandardScaler
from datetime import timedelta 
from datetime import datetime
from numpy.linalg import LinAlgError

/anaconda/envs/py35/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [12]:
df_data = pd.read_csv('full_diff_data.csv', parse_dates=['time_stamp', 'time_stamp_local'])
df_data.set_index('time_stamp_local', inplace = True)

In [13]:
df_data.shape

(8760, 10)

In [14]:
df_data.head(24)

,time_stamp,price,price_24hr_ago,price_diff,load,load_24hr_ago,load_diff,temp,temp_24hr_ago,temp_diff
time_stamp_local,,,,,,,,,,
2017-01-01 00:00:00,2017-01-01 05:00:00,32.28,35.47,-3.19,5011.2,5128.0,-116.8,46.0,35.0,11.0
2017-01-01 01:00:00,2017-01-01 06:00:00,30.77,34.08,-3.31,4837.3,4896.2,-58.9,45.0,35.0,10.0
2017-01-01 02:00:00,2017-01-01 07:00:00,31.17,27.90,3.27,4652.6,4733.9,-81.3,46.0,35.0,11.0
2017-01-01 03:00:00,2017-01-01 08:00:00,32.00,28.09,3.91,4500.7,4640.7,-140.0,46.0,35.0,11.0
2017-01-01 04:00:00,2017-01-01 09:00:00,27.89,27.20,0.69,4423.5,4610.1,-186.6,47.0,34.5,12.5
2017-01-01 05:00:00,2017-01-01 10:00:00,28.23,29.61,-1.38,4433.2,4678.2,-245.0,47.0,34.0,13.0
2017-01-01 06:00:00,2017-01-01 11:00:00,34.64,32.50,2.14,4486.5,4840.1,-353.6,47.0,34.0,13.0
2017-01-01 07:00:00,2017-01-01 12:00:00,25.44,33.46,-8.02,4526.2,5015.0,-488.8,47.0,34.0,13.0
2017-01-01 08:00:00,2017-01-01 13:00:00,22.86,34.35,-11.49,4645.0,5248.1,-603.1,47.0,35.0,12.0


In [15]:
df_data.tail(2)

,time_stamp,price,price_24hr_ago,price_diff,load,load_24hr_ago,load_diff,temp,temp_24hr_ago,temp_diff
time_stamp_local,,,,,,,,,,
2017-12-31 22:00:00,2018-01-01 03:00:00,135.27,109.81,25.46,6467.6,6304.0,163.6,11.0,22.5,-11.5
2017-12-31 23:00:00,2018-01-01 04:00:00,143.23,98.40,44.83,6263.7,6014.8,248.9,10.0,22.0,-12.0


In [16]:
#df_data.price.resample('1D').apply(sum)

In [17]:
def saving_byday(df_price):
    df_tmp = df_price.sort_values()
    return df_tmp.values[df_tmp.shape[0]-2:].sum() - df_tmp.values[:2].sum()

In [18]:
df_data.price.resample('1D').apply(saving_byday)

time_stamp_local
2017-01-01     70.30
2017-01-02    236.32
2017-01-03     47.78
2017-01-04     78.61
2017-01-05     95.87
2017-01-06     44.60
2017-01-07    157.20
2017-01-08    104.50
2017-01-09    197.33
2017-01-10    257.72
2017-01-11     45.53
2017-01-12     88.47
2017-01-13     90.62
2017-01-14     52.67
2017-01-15    111.00
2017-01-16    105.28
2017-01-17    188.31
2017-01-18     72.77
2017-01-19    141.10
2017-01-20     25.65
2017-01-21     31.94
2017-01-22     52.27
2017-01-23     67.70
2017-01-24      9.36
2017-01-25    111.98
2017-01-26     99.06
2017-01-27    127.68
2017-01-28     30.84
2017-01-29     30.58
2017-01-30    119.44
               ...  
2017-12-02    100.47
2017-12-03    245.46
2017-12-04     34.66
2017-12-05     78.56
2017-12-06    218.19
2017-12-07    142.97
2017-12-08    142.43
2017-12-09    168.48
2017-12-10     37.87
2017-12-11    109.71
2017-12-12     41.75
2017-12-13    100.76
2017-12-14    167.61
2017-12-15    523.99
2017-12-16    118.96
2017-12-17    112

In [19]:
df_data.price.resample('1D').apply(saving_byday).sum()/2

19032.029999999999

In [20]:
df_data.price.resample('1D').apply(saving_byday).sum()/df_data.price.sum()

0.13485748313832394